In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
import streamlit as st
import time
import base64
import uuid
import tempfile
from langchain_upstage import UpstageEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader

from langchain_upstage import ChatUpstage
from langchain_core.messages import HumanMessage, SystemMessage

In [2]:
chat = ChatUpstage(upstage_api_key=os.getenv("UPSTAGE_API_KEY"))

messages = [
  SystemMessage(content="You are a helpful assistant."),
  HumanMessage(content="Hi, how are you?")
]
response = chat.invoke(messages)
print(response.content)

Oh, hey there! I'm doing great, thanks for asking. Just been a bit busy with work and stuff. How about you?


### PDF 파일 업로드

In [ ]:
# from langchain.document_loaders import PyPDFLoader

from langchain_community.document_loaders import PyPDFLoader

In [4]:
loader = PyPDFLoader("../pdfs/EU-탄소중립산업법-주요-내용.pdf")


In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
    )

pages = loader.load_and_split(text_splitter)

pages[:5]

[Document(page_content='1. 도입 배경\n2021.12.8, EU집행위는 제3국의 경제적 위협 으로부터 유럽연합 (EU) 및 회원국을 보호하기 위해 통상위협대응 규정 (안)을 \n발표했습니다 . EU집행위는 EU가 제3국의 경제적 위협에 신속히 대응하고 필요시 최후의 수단으로 대응조치\n(countermeasure) 를 취하여 EU와 회원국의 경제적 이익을 보호 하기 위해 본 규정 (안)을 준비했습니다 . (제1조) \n본 규정 (안)은 지정학적 긴장이 고조되며 , 통상이 점점 더 무기화되어 가고 있는 시점에 EU가 통상 정책을 강화하고 , 제3국의 \n경제적 위협에 효과적으로 대응하기 위한 해결방안으로 제시한 것입니다 . 2018.3 미국이 철강 232 조 관세를 부과함에 따라 \nEU는 직접적인 피해를 입었습니다 . 또한 , 2019. 12 세계무역기구 TO) 상소기구가 마비되어 WTO 분쟁해결의 효율성이 약 없\n게 되어 , EU는 경제적 위협에 더욱 취약해졌습니다 .\n본 규정 (안)의 도입은 2019.12 집행위가 통상권한집행 규정 (Trade Enforcement Regulation) 을 개정하기 위한 입법안을 \n제시한 후 처음 논의가 되었습니다 . 당시 유럽의회는 경제적 위협에 대한 우려를 제기했지만 , 개정된 통상권한집행 규정은 EU가 \n경제적 위협에 대응할 수 있는 방법을 규정하지 않았습니다 . 그 후속조치로 집행위 , 의회 , 이사회는 2021 년 말 까지 집행위가 경\n제적 위협에 대응하기 위한 별도의 입법안을 마련하겠다는 것을 명시한 공동 선언문을 채택했습니다 .\n그 결과 이번에 발표된 통상위협대응 규정 (안)은 제3국의 경제적 위협에 대응하기 위해 EU가 취할 조치를 2단계로 명시하고 있\n습니다 . 첫째 , 집행위는 제3국의 경제적 위협을 분석하여 , 그 위협의 유무를 공식적으로 결정한 후, 경제적 위협이 중단될 수 있\n도록 제3국과 협의를 합니다 . 둘째 , 제3국과의 협의가 경제적 위협을 중단하는 결과를 

In [8]:
from langchain.embeddings import OpenAIEmbeddings
from langchain_upstage import UpstageEmbeddings

from langchain.vectorstores import Chroma

directory = "./pdf_db"       # 경로 지정 -> 크로마 함수에서 폴더 만들어 줄것임


vectorstore = Chroma.from_documents(
                        pages, 
                        UpstageEmbeddings(model="solar-embedding-1-large"),
                        persist_directory=directory # persists the vectors to the file system
                        )


vectorstore.persist()

c:\Users\User\anaconda3\envs\llm\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [9]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import os 
from langchain.chains import RetrievalQA

# 검색기 설정 
retriever = vectorstore.as_retriever(
                                search_type="similarity", # Cosine Similarity
                                search_kwargs={"k": 3,} 
                                
)


In [10]:
# QA 체인 설정
qa = RetrievalQA.from_llm(llm=chat, retriever=retriever, return_source_documents=True)

In [11]:
query = '탄소중립 전략 프로젝트에 대해서 알려줘.'
result = qa(query)          # 질문 -> qa -> answer

print(result["result"])     # 대답 출력 

c:\Users\User\anaconda3\envs\llm\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


탄소중립 전략 프로젝트는 EU가 지정한 8개의 탄소중립 전략기술(태양광 발전 및 태양열, 육상 및 해상 풍력발전, 배터리 및 저장, 히트펌프 및 지열에너지, 전해조 및 수소연료전지, 지속가능 바이오가스 및 바이오메탄, 탄소 포집 저장(CCUS), 그리드)의 역내 제조 역량 및 EU 에너지시스템 회복력 강화에 기여하는 프로젝트와 역내 소재 CO2 저장 프로젝트를 말합니다. 이러한 프로젝트는 '탄소중립 전략 프로젝트(Net-zero Strategic Project)'로 선정됩니다.


### 메세지 기억하고 대답 근거 출력하기

In [16]:
# 1) 챗봇에 '기억'을 입히기 위한 첫번째 단계 

# 이전의 메시지들과 최신 사용자 질문을 분석해, 문맥에 대한 정보가 없이 혼자서만 봤을때 이해할 수 있도록 질문을 다시 구성함
# 즉 새로 들어온 그 질문 자체에만 집중할 수 있도록 다시 재편성
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = """이전 대화 내용과 최신 사용자 질문이 있을 때, 이 질문이 이전 대화 내용과 관련이 있을 수 있습니다. 
이런 경우, 대화 내용을 알 필요 없이 독립적으로 이해할 수 있는 질문으로 바꾸세요. 
질문에 답할 필요는 없고, 필요하다면 그저 다시 구성하거나 그대로 두세요."""

# MessagesPlaceholder: 'chat_history' 입력 키를 사용하여 이전 메세지 기록들을 프롬프트에 포함시킴.
# 즉 프롬프트, 메세지 기록 (문맥 정보), 사용자의 질문으로 프롬프트가 구성됨. 
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# 이를 토대로 메세지 기록을 기억하는 retriever를 생성합니다.
history_aware_retriever = create_history_aware_retriever(
    chat, retriever, contextualize_q_prompt
)


In [17]:
# 2) 두번째 단계로, 방금 전 생성한 체인을 사용하여 문서를 불러올 수 있는 retriever 체인을 생성합니다.
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """질문-답변 업무를 돕는 보조원입니다. 
질문에 답하기 위해 검색된 내용을 사용하세요. 
답을 모르면 모른다고 말하세요. 
답변은 세 문장 이내로 간결하게 유지하세요.

## 답변 예시
📍답변 내용: 
📍증거: 

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(chat, qa_prompt)

# 결과값은 input, chat_history, context, answer 포함함.
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


In [19]:
history_messages = []

In [20]:
prompt = input()
# 탄소중립플랫폼(Net-Zero Europe Platform) 설립

history_messages.append(prompt)


In [21]:
result = rag_chain.invoke({"input": prompt, "chat_history": history_messages})


In [22]:
result["context"]

[Document(page_content='6장. 혁신26조 탄소중립 규제샌드 박스\n27조 중소기업을 위한 조치\n7장. 거버넌스28조 탄소중립 유럽 플랫폼 설립 및 과업\n29조 탄소중립 유럽 플랫폼 구조 및 기능\n30조 국가 에너지 ·기후계획과의 조율\n8장 모니터링 31조 모니터링\n9장 최종조항32조 권한 위임\n33조 위임권 행사\n34조 위원회 절차\n35조 평가\n36조 기밀정보 처리\n37조 Regulation (EU) 2018/1724(Single Digital Gateway) 개정\n38조 발효 및 시행', metadata={'page': 7, 'source': '../pdfs/EU-탄소중립산업법-주요-내용.pdf'}),
 Document(page_content='6장. 혁신26조 탄소중립 규제샌드 박스\n27조 중소기업을 위한 조치\n7장. 거버넌스28조 탄소중립 유럽 플랫폼 설립 및 과업\n29조 탄소중립 유럽 플랫폼 구조 및 기능\n30조 국가 에너지 ·기후계획과의 조율\n8장 모니터링 31조 모니터링\n9장 최종조항32조 권한 위임\n33조 위임권 행사\n34조 위원회 절차\n35조 평가\n36조 기밀정보 처리\n37조 Regulation (EU) 2018/1724(Single Digital Gateway) 개정\n38조 발효 및 시행', metadata={'page': 7, 'source': '../pdfs/EU-탄소중립산업법-주요-내용.pdf'}),
 Document(page_content='- 3 -\n6소비자 인센티브 지급 및 숙련인력 육성\n ㅇ 회원국 , 지역 , 공공법에 적용받는 기관은 탄소중립기술 최종제품의 구매 촉진 지원책을 도입할 수 있고 \n인센티브는 탄소중립 최종제품 소비자가격의 5%를 초과할 수 없음\n ㅇ 유럽탄소중립산업 아카데미 (European Net-Zero Industry Academy) 를 설립 , 탄소중립기술 개발 , 생산 , \n운영 , 재활용 등과 관련된 교육 훈련 프로그램 개발 및 운영 , 

In [23]:
result["answer"]

'탄소중립플랫폼(Net-Zero Europe Platform) 설립은 탄소중립 유럽법(EU Taxonomy Regulation)의 7장에 해당하는 내용입니다. 이 플랫폼은 탄소중립 기술 분야로의 투자와 제조 목표 달성을 위한 회원국의 프로젝트 조정, 국가 간 소통 강화, 제3국과의 파트너십 구축, 기술 인력 양성 등의 전반적인 모니터링과 리포팅을 담당합니다. 이를 통해 탄소중립 기술의 개발과 상용화를 촉진하고, 유럽 내 탄소중립 경제로의 전환을 위한 노력을 조정하고 강화하는 역할을 합니다.'